Quantconnect also allows to trade futures

You can request Future data by using **AddFuture(symbol, resolution)** which adds a future chain to the security list.<br />

Possible resolutions are *tick, second & minute*

The default expiration date for futures is 35 days.<br />
A different limit can be defined by using **future.SetFilter(min_expiration_date, max_expiration_date)**

As usual the Futures can be accessed in the OnData function.<br />
To access all contracts you can loop over all contracts in the following way:

```python
for chain in slice.FutureChains.Values:  # Loop over the future chains
        contracts = chain.Contracts  # Get all contracts in the current chain
        for contract in contracts.Values:  # Loop over the contracts
            # do something
```

## Buying futures
Let's buy our first futures, an oil future called WTI Crude Oil (Futures.Energies.CrudeOilWTI) and a golf future called Futures.Metals.Gold. <br />
You can find a list of all futures here: https://www.quantconnect.com/docs/data-library/futures#Futures-Reference-Tables

In fact we will filter all contracts with an expiration date between 0 and 75 days.<br />
Then we simply invest 50% in the first contract of both future chains.<br />
However those contracts are sorted by the ticker name and not the date, thus the first contract may seem kind of random - In the next cell you will see how to sort and filter those contracts.

In [ ]:
class OilAndGoldFuture(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 30)
        self.SetEndDate(2010, 2, 5)

        self.SetCash(10000)

        self.contractSymbol = None

        # Subscribe to the oil future  
        oil_future = self.AddFuture(Futures.Energies.CrudeOilWTI, Resolution.Minute)
        
        # Subscribe to the gold future  
        gold_future = self.AddFuture(Futures.Metals.Gold, Resolution.Minute)


        # Take all futures which expire within 0-75 days
        oil_future.SetFilter(0, 75)
        gold_future.SetFilter(0, 75)

    def OnData(self, slice):
        
        # Log all Future chains - /CL for Oil and /GC for gold
        for chain in slice.FutureChains.Values:
            self.Log(f"Chain:{chain}")
            
            # Show all stored information in the contracts
            for contract in chain.Contracts:
                self.Log(f"Contract:{contract}")
                
            # Loop over the contract.Values (the ticker) and print the value and the symbol
            for contract_value in chain.Contracts.Values:
                self.Log(f"Value: {contract_value}, Symbol: {contract_value.Symbol}")



        # Now we invest 50% in the first oil future and 50% in the first gold future
        if not self.Portfolio.Invested:
            for chain in slice.FutureChains.Values:  # Loop over the futures
                contracts = chain.Contracts  # Get the corresponding contracts
                for contract in contracts.Values:  # Loop over the contracts
                    
                    self.Log(f"Expiration Date: {contract}: {contract.Expiry}")

                    self.SetHoldings(contract.Symbol, 0.5)  # Invest 50% in the first contract
                    break
                

## Selecting a contract

Each future chain contains multiple contracts. <br />
We can filter and sort the contracts based on the following properties: (https://www.quantconnect.com/docs/data-library/futures#Futures-Using-Futures-Data)

* Expiry: (datetime) When the future expires
* OpenInterest: (decimal) Number of open interest.
* LastPrice: (decimal) Last sale price.
* Volume:  (long) reported volume.
* BidPrice: (decimal) bid quote price.
* BidSize: (long) bid quote size.
* AskPrice: (decimal) ask quote price.
* AskSize: (long) ask quote size.

Let's buy the contract with the highest OpenInterest and an AskPrice > 75

In [ ]:
class OilFuture(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 30)
        self.SetEndDate(2010, 2, 5)

        self.SetCash(10000)

        self.contractSymbol = None

        # Subscribe to the Futurechains
        oil_future = self.AddFuture(Futures.Energies.CrudeOilWTI, Resolution.Minute)

        # Take all futures which expire within 0-365 days
        oil_future.SetFilter(0, 365)


    def OnData(self, slice):
        
        if not self.Portfolio.Invested:  # Only invest if currently not invested
            for chain in slice.FutureChains.Values:  # Loop over the futures
                
                self.Log(f"How many contracts: {len(chain.Contracts)}")
                
                # Filter all contracts where the AskPrice is > 75
                contracts_filtered = [contract for contract in chain if contract.AskPrice > 75]
                self.Log(f"How many contracts filtered: {len(contracts_filtered)}")

                # Sort by OpenInterest
                sorted_popular_contracts = sorted(contracts_filtered, key=lambda x: x.OpenInterest, reverse=True)

                
                contract_to_buy = sorted_popular_contracts[0].Symbol  # Get the most popular contract and invest everything
                self.Log(f"Buying: {contract_to_buy}")
                self.SetHoldings(contract_to_buy, 1)
                return
        

## Computing the notional value
We can computing the notional value by accessing the current AskPrice and multiply this value with the **ContractMultiplier** which is defined in **future.SymbolProperties.ContractMultiplier**
Let's plot the notional value minutewise by removing the **IsInvested** check at the beginning of *OnData*

Additionally we log the remaining margin of our portfolio **self.Portfolio.MarginRemaining**

In [ ]:
class OilFuture(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 30)
        self.SetEndDate(2010, 2, 5)

        self.SetCash(10000)

        self.contractSymbol = None

        # Subscribe to the Futurechains
        self.oil_future = self.AddFuture(Futures.Energies.CrudeOilWTI, Resolution.Minute)

        # Take all futures which expire within 0-100 days
        self.oil_future.SetFilter(0, 100)
        price_plot = Chart('Oilprice - Notional')
        price_plot.AddSeries(Series('Price', SeriesType.Line, 0))
        self.AddChart(price_plot)  # Add the Chart to your environment


    def OnData(self, slice):
        
        for chain in slice.FutureChains.Values:  # Loop over the futures

            sorted_popular_contracts = sorted(chain, key=lambda x: x.OpenInterest, reverse=True)
            self.Log(f"Num contracts: {len(sorted_popular_contracts)}")
            contract_to_buy = sorted_popular_contracts[0]  # Get the most popular contract

            
            notionalValue = contract_to_buy.AskPrice*self.oil_future.SymbolProperties.ContractMultiplier 
            self.Debug(notionalValue)
            self.Log(f"Buying: {contract_to_buy.Symbol}")
            self.Log(self.Portfolio.MarginRemaining)
            self.SetHoldings(contract_to_buy.Symbol, 1)

            self.Plot("Oilprice - Notional", "Price", notionalValue)
            return

        
        

## Trading equities based on Futures
We can buy Shell if oil futures rise and sell Shell if they drop in value

In [ ]:
class OilFutureAndShell(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 1)
        self.SetEndDate(2015, 12, 31)

        self.SetCash(10000)

        self.contractSymbol = None

        # Subscribe to the Futurechains
        self.oil_future = self.AddFuture(Futures.Energies.CrudeOilWTI, Resolution.Minute)
        self.shell = self.AddEquity("RDS.A", Resolution.Daily)

        self.future_price = 0  #Store the current price of the oil future
        
        # Take all futures which expire within 0-100 days
        self.oil_future.SetFilter(0, 100)
        price_plot = Chart('Oilprice - Notional')
        price_plot.AddSeries(Series('Price', SeriesType.Line, 0))
        self.AddChart(price_plot)  # Add the Chart to your environment
        

    def OnData(self, slice):
        
        
        for chain in slice.FutureChains.Values:  # Loop over the futures
        
            sorted_popular_contracts = sorted(chain, key=lambda x: x.OpenInterest, reverse=True)
            contract_to_buy = sorted_popular_contracts[0]  # Get the most popular contract
            
            # Store ask price of most popular future contract
            self.future_price = contract_to_buy.AskPrice
            break
        
        # If the oil price is higher than 60$ we buy Shell
        if self.future_price > 60 and not self.Portfolio.Invested:
            self.Log("Invest")
            self.SetHoldings(self.shell.Symbol, 1)
        
        # Else we sell everything
        elif self.future_price < 60:
            self.Log("Sell")
            self.Liquidate()

        
        